In [46]:
import os
os.environ['YELP_API_KEY']='<INSERT YELP API KEY HERE>'

In [47]:
import tecton
from datetime import datetime
import pandas as pd

In [48]:
tecton.login("community.tecton.ai")

Please visit the following link to login and access the authentication code:
https://login.tecton.ai/oauth2/default/v1/authorize?response_type=code&client_id=0oawb0ilshCmsYdah357&redirect_uri=https%3A%2F%2Fwww.tecton.ai%2Fauthorization-callback&state=1313747145222797495&scope=openid+offline_access+profile+email&code_challenge_method=S256&code_challenge=KF-JASccxvjfE2nUjFw4euFYfBYpCJNZA_pA57nxoCo


Paste the authentication code here: C3YKZHOpSSeVnT4DGviKYbqy_a3LBpvqTjc8sD9L3lA


✅ Authentication successful!


In [49]:
from tecton import Entity

user = Entity(
    name='user_id',
    join_keys=['user_id'],
    description='A user on the platform',
)

user.validate()

Entity 'user_id': Successfully validated.


In [50]:
from tecton import BatchSource, FileConfig

user_historic_trips_batch = BatchSource(
        name='user_historic_trips',
        batch_config=FileConfig(
            uri='s3://travel-data-public-demo/user_historic_trips.parquet',
            file_format='parquet',
            timestamp_field='transaction_timestamp'
    )
)

user_historic_trips_batch.validate()


BatchSource 'user_historic_trips': Successfully validated.


In [51]:
user_historic_trips_batch.get_dataframe().to_pandas()

---- Executing stage: Reading data sources                               00:01


,transaction_timestamp,user_id,departure_date,return_date,origin,origin_name,destination,destination_name
0,2023-08-21 13:34:07.420082,8iovlkic,2023-10-08 13:34:07.420082,2023-10-24 13:34:07.420082,DKK,Chautauqua County/Dunkirk Airport,PTW,Heritage Field
1,2023-07-01 13:34:07.420082,qhr6omop,2023-07-20 13:34:07.420082,2023-08-04 13:34:07.420082,YHT,Haines Junction Airport,YUT,Repulse Bay Airport
2,2023-07-01 13:34:07.420082,97b4q3of,2023-07-29 13:34:07.420082,2023-08-01 13:34:07.420082,NAJ,Nakhchivan Airport,BKD,Stephens County Airport
3,2023-05-10 13:34:07.420082,9tb07o5m,2023-06-05 13:34:07.420082,2023-06-11 13:34:07.420082,ZPB,Sachigo Lake Airport,HPH,Cat Bi International Airport
4,2023-08-18 13:34:07.420082,hvofy0tc,2023-09-11 13:34:07.420082,2023-10-02 13:34:07.420082,DDG,Dandong Airport,HLC,Hill City Municipal Airport
...,...,...,...,...,...,...,...,...
49991,2023-06-27 13:34:27.786947,sse3bici,2023-08-12 13:34:27.786947,2023-08-15 13:34:27.786947,GOK,Guthrie/Edmond Regional Airport,ARH,Talagi Airport
49992,2023-05-31 13:34:27.786947,e9i7dq6q,2023-07-18 13:34:27.786947,2023-08-12 13:34:27.786947,PMB,Pembina Municipal Airport,NCA,North Caicos Airport
49993,2023-07-09 13:34:27.786947,vhv1al6l,2023-08-28 13:34:27.786947,2023-08-29 13:34:27.786947,IGN,Iligan Airport,TOF,Bogashevo Airport
49994,2023-05-25 13:34:27.786947,o343f0jp,2023-06-04 13:34:27.786947,2023-06-20 13:34:27.786947,VYD,Vryheid Airport,PSL,Perth/Scone Airport


In [52]:
from tecton import batch_feature_view, Aggregation, FilteredSource, DeltaConfig
from tecton.aggregation_functions import last
from datetime import datetime, timedelta
from tecton.types import Float64, Field, Bool, String, Int64, Timestamp

@batch_feature_view(
    sources=[FilteredSource(user_historic_trips_batch)],
    entities=[user],
    mode='pandas',
    online=True,
    offline=True,
    feature_start_time=datetime(2023,1, 1),
    batch_schedule=timedelta(days=1),
    description='Last 5 trips the user completed in the last 365 days',
    timestamp_field='return_date', #notice we set the timestamp to the return date; that is the trip completion date
    schema=[Field("user_id", String), 
            Field("destination_name", String), 
            Field("return_date", Timestamp)],
    aggregation_interval=timedelta(days=1),
    offline_store = DeltaConfig(
                    time_partition_size=timedelta(days=1),
                    subdirectory_override=None),
    aggregations = [Aggregation(column="destination_name", function=last(5), time_window=timedelta(days=365))]
)

def user_last_trip_completed_fv(user_historic_trips_batch):
    return user_historic_trips_batch[['user_id', 'destination_name', 'return_date']]

user_last_trip_completed_fv.validate()


BatchFeatureView 'user_last_trip_completed_fv': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'user_last_trip_completed_fv': Successfully validated.
BatchFeatureView 'user_last_trip_completed_fv': Successfully validated.


In [53]:
from tecton import FeatureService

user_feature_service = FeatureService(
    name="user_service:v1",
    features=[user_last_trip_completed_fv]
)

user_feature_service.validate()

FeatureService 'user_service:v1': Successfully validated.


In [55]:
event = {"user_id": "demouser123",
         "timestamp": datetime.now()}
training_event = pd.DataFrame([event])

user_feature_service.get_historical_features(training_event, from_source=True).to_pandas()

---- Executing stage: Reading data sources                               00:01
---- Executing stage: Evaluating feature view pipelines                  00:00
---- Executing stage: Computing aggregated features & joining results    00:00
---- Executing stage: Computing ODFV features                            00:00


,user_id,timestamp,user_last_trip_completed_fv__destination_name_last_5_365d_1d
0,demouser123,2023-11-16 09:15:49.177591,[Narita International Airport]


In [56]:
user_home_airport_batch = BatchSource(
        name='user_home_airport',
        batch_config=FileConfig(
            uri='s3://travel-data-public-demo/user_home_airport.parquet',
            file_format='parquet',
            timestamp_field='last_updated_timestamp'
    )
)
user_home_airport_batch.validate()
user_home_airport_batch.get_dataframe().to_pandas()

BatchSource 'user_home_airport': Successfully validated.
---- Executing stage: Reading data sources                               00:00


,last_updated_timestamp,user_id,first_name,last_name,home_airport_code,home_airport_name
0,2023-11-08 13:25:32.140615,8iovlkic,Allison,Bishop,DKK,Chautauqua County/Dunkirk Airport
1,2023-11-08 13:25:32.140633,qhr6omop,Courtney,Cole,YHT,Haines Junction Airport
2,2023-11-08 13:25:32.140635,97b4q3of,Cynthia,Mccann,NAJ,Nakhchivan Airport
3,2023-11-08 13:25:32.140638,9tb07o5m,Matthew,Johnson,ZPB,Sachigo Lake Airport
4,2023-11-08 13:25:32.140640,hvofy0tc,Susan,Jackson,DDG,Dandong Airport
...,...,...,...,...,...,...
9995,2023-11-08 13:25:32.157640,sse3bici,Heather,Arias,GOK,Guthrie/Edmond Regional Airport
9996,2023-11-08 13:25:32.157641,e9i7dq6q,Kyle,Chan,PMB,Pembina Municipal Airport
9997,2023-11-08 13:25:32.157642,vhv1al6l,Stefanie,Kennedy,IGN,Iligan Airport
9998,2023-11-08 13:25:32.157644,o343f0jp,Terry,Rodriguez,VYD,Vryheid Airport


In [57]:
@batch_feature_view(
    sources=[user_home_airport_batch],
    entities=[user],
    mode='pandas',
    online=True,
    offline=True,
    feature_start_time=datetime(2023,1, 1),
    batch_schedule=timedelta(days=1),
    description='User home airport',
    timestamp_field='last_updated_timestamp', 
    offline_store = DeltaConfig(
    time_partition_size=timedelta(days=1),
    subdirectory_override=None),
    schema=[Field("user_id", String), 
                            Field("home_airport_code", String), 
                            Field("home_airport_name", String), 
                            Field("last_updated_timestamp", Timestamp)]
)
def user_home_airport_fv(user_home_airport_batch):
    return user_home_airport_batch[['user_id', 'home_airport_code', 'home_airport_name', 'last_updated_timestamp']]

user_home_airport_fv.validate()

BatchFeatureView 'user_home_airport_fv': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'user_home_airport_fv': Successfully validated.
BatchFeatureView 'user_home_airport_fv': Successfully validated.


In [58]:
user_likes_batch = BatchSource(
        name='user_likes_batch',
        batch_config=FileConfig(
            uri='s3://travel-data-public-demo/user_likes.parquet',
            file_format='parquet',
            timestamp_field='last_updated_timestamp'
    )
)
user_likes_batch.validate()
user_likes_batch.get_dataframe().to_pandas()

BatchSource 'user_likes_batch': Successfully validated.
---- Executing stage: Reading data sources                               00:00


,user_id,favorite_activity,last_updated_timestamp
0,8iovlkic,"[Food, Snorkeling]",2023-11-08 13:25:45.718667
1,qhr6omop,"[Running, Bicycling, Food]",2023-11-08 13:25:45.718667
2,97b4q3of,[Bicycling],2023-11-08 13:25:45.718667
3,9tb07o5m,"[Snorkeling, Running]",2023-11-08 13:25:45.718667
4,hvofy0tc,[Sightseeing],2023-11-08 13:25:45.718667
...,...,...,...
9995,sse3bici,"[Bicycling, Shopping, Snorkeling]",2023-11-08 13:25:45.718667
9996,e9i7dq6q,[Sightseeing],2023-11-08 13:25:45.718667
9997,vhv1al6l,[Running],2023-11-08 13:25:45.718667
9998,o343f0jp,"[Food, Sightseeing]",2023-11-08 13:25:45.718667


In [59]:
from tecton import batch_feature_view
from datetime import datetime, timedelta
from tecton.types import Float64, Field, Bool, String, Int64, Timestamp, Array

@batch_feature_view(
    sources=[user_likes_batch],
    entities=[user],
    mode='pandas',
    online=True,
    offline=True,
    feature_start_time=datetime(2023,1, 1),
    batch_schedule=timedelta(days=1),
    offline_store = DeltaConfig(
    time_partition_size=timedelta(days=1),
    subdirectory_override=None),
    description='User favorite activities on their profile',
    timestamp_field='last_updated_timestamp', 
    schema=[Field("user_id", String), 
                            Field("last_updated_timestamp", Timestamp), 
                            Field("favorite_activity", Array(String))]
)
def user_likes_fv(user_likes_batch):
    return user_likes_batch[['user_id', 'favorite_activity', 'last_updated_timestamp']]

user_likes_fv.validate()


BatchFeatureView 'user_likes_fv': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'user_likes_fv': Successfully validated.
BatchFeatureView 'user_likes_fv': Successfully validated.


In [60]:
from tecton import FeatureService

user_feature_service2 = FeatureService(
    name="user_service:v2",
    features=[user_home_airport_fv,
             user_likes_fv,
             user_last_trip_completed_fv]
)

user_feature_service2.validate()

FeatureService 'user_service:v2': Successfully validated.


In [61]:
event = {"user_id": "demouser123",
         "timestamp": datetime.now()}
training_event = pd.DataFrame([event])

user_feature_service2.get_historical_features(training_event, from_source=True).to_pandas()

---- Executing stage: Reading data sources                               00:02
---- Executing stage: Evaluating feature view pipelines                  00:00
---- Executing stage: Computing aggregated features & joining results    00:00

---- Executing stage: Computing ODFV features                            00:00

,user_id,timestamp,user_home_airport_fv__home_airport_code,user_home_airport_fv__home_airport_name,user_likes_fv__favorite_activity,user_last_trip_completed_fv__destination_name_last_5_365d_1d
0,demouser123,2023-11-16 09:18:07.867882,SFO,San Francisco International Airport,"[Bicycling, Running, Food]",[Narita International Airport]


In [ ]:
#Streaming Time

In [62]:
from tecton import PushSource
from tecton.types import String, Int64, Timestamp, Field

input_schema = [
    Field(name="transaction_timestamp", dtype=Timestamp),
    Field(name="user_id", dtype=String),
    Field(name="departure_date", dtype=Int64),
    Field(name="return_date", dtype=Int64),
    Field(name="destination_name", dtype=String),
    Field(name="origin_name", dtype=String),
]

user_searches_eventsource = PushSource(
    name="user_searches_stream",
    schema=input_schema,
    batch_config=FileConfig(
            uri='s3://travel-data-public-demo/future_trip_searches.parquet',
            file_format='parquet',
            timestamp_field='transaction_timestamp'
    ),
    description="User flight searches stream",
)

user_searches_eventsource.validate()

PushSource 'user_searches_stream': Deriving schema.
PushSource 'user_searches_stream': Successfully validated.


In [63]:
user_searches_eventsource.get_dataframe().to_pandas()

---- Executing stage: Reading data sources                               00:01


,transaction_timestamp,user_id,departure_date,return_date,destination_name,origin_name
0,2023-10-25 14:04:20.972784,8iovlkic,2024-03-04 14:04:20.908761,2024-03-28 14:04:20.908761,Eastland Municipal Airport,Chautauqua County/Dunkirk Airport
1,2023-11-09 14:04:20.974419,qhr6omop,2024-01-14 14:04:20.908761,2024-01-29 14:04:20.908761,Marion Municipal Airport,Haines Junction Airport
2,2023-09-22 14:04:20.975721,97b4q3of,2024-04-09 14:04:20.908761,2024-04-22 14:04:20.908761,Kochi Ryoma Airport,Nakhchivan Airport
3,2023-10-02 14:04:20.977899,9tb07o5m,2024-04-10 14:04:20.908761,2024-05-09 14:04:20.908761,Saint John Airport,Sachigo Lake Airport
4,2023-09-19 14:04:20.980752,hvofy0tc,2024-01-28 14:04:20.908761,2024-02-12 14:04:20.908761,Miyakejima Airport,Dandong Airport
...,...,...,...,...,...,...
9995,2023-09-30 14:04:26.345124,sse3bici,2024-05-06 14:04:20.908761,2024-06-04 14:04:20.908761,Munda Airport,Guthrie/Edmond Regional Airport
9996,2023-10-04 14:04:26.345619,e9i7dq6q,2024-05-05 14:04:20.908761,2024-05-11 14:04:20.908761,Viracopos International Airport,Pembina Municipal Airport
9997,2023-11-12 14:04:26.346113,vhv1al6l,2023-12-24 14:04:20.908761,2024-01-08 14:04:20.908761,Greeneville Municipal Airport,Iligan Airport
9998,2023-10-22 14:04:20.908761,o343f0jp,2023-12-16 14:04:20.908761,2024-01-03 14:04:20.908761,Canobie Airport,Vryheid Airport


In [64]:
from tecton import stream_feature_view, BatchTriggerType, DeltaConfig
from datetime import datetime, timedelta
from tecton.types import Field, Timestamp, Int64, String

@stream_feature_view(
    source=user_searches_eventsource,
    entities=[user],
    mode='python',
    online=True,
    offline=True,
    schema = input_schema,
    feature_start_time=datetime(2023,1,1),
    description='Return the last trip search made by a user on the platform',
    timestamp_field="transaction_timestamp",
    manual_trigger_backfill_end_time=datetime(2023,10,31),
    batch_trigger = BatchTriggerType.MANUAL,
    batch_schedule=timedelta(days=1), 
    offline_store = DeltaConfig(
                    time_partition_size=timedelta(days=1),
                    subdirectory_override=None)
)
def user_flight_search_fv(user_searches_eventsource):
    user_searches_eventsource['departure_date'] = user_searches_eventsource['departure_date'].astype(str).str.split().str[0]
    user_searches_eventsource['return_date'] = user_searches_eventsource['return_date'].astype(str).str.split().str[0]
    return user_searches_eventsource

user_flight_search_fv.validate()

StreamFeatureView 'user_flight_search_fv': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'user_flight_search_fv': Successfully validated.
StreamFeatureView 'user_flight_search_fv': Successfully validated.


In [65]:
user_feature_service3 = FeatureService(
    name="user_service:v3",
    features=[user_home_airport_fv,
              user_likes_fv,
              user_last_trip_completed_fv,
              user_flight_search_fv]
)

user_feature_service3.validate()

FeatureService 'user_service:v3': Successfully validated.


In [66]:
event = {"user_id": "demouser123",
         "timestamp": datetime.now()}
training_event = pd.DataFrame([event])

user_feature_service3.get_historical_features(training_event, from_source=True).to_pandas()

---- Executing stage: Reading data sources                               00:04
---- Executing stage: Evaluating feature view pipelines                  00:00
---- Executing stage: Computing aggregated features & joining results    00:00

---- Executing stage: Computing ODFV features                            00:00

,user_id,timestamp,user_home_airport_fv__home_airport_code,user_home_airport_fv__home_airport_name,user_likes_fv__favorite_activity,user_last_trip_completed_fv__destination_name_last_5_365d_1d,user_flight_search_fv__origin_name,user_flight_search_fv__return_date,user_flight_search_fv__departure_date,user_flight_search_fv__destination_name
0,demouser123,2023-11-16 09:25:43.413699,SFO,San Francisco International Airport,"[Bicycling, Running, Food]",[Narita International Airport],San Francisco International Airport,2023-12-27,2023-12-13,Singapore Changi Airport


In [ ]:
#ODFV Time

In [67]:
def yelp_get_hotels(company_address, api_key):
    import json
    import time
    import urllib.request, urllib.error, urllib.parse

    API_KEY = api_key
    ADDRESS = company_address

    URL = "https://api.yelp.com/v3/businesses/search"
    PARAMS = {
        "term": "hotel",
        "location": ADDRESS,
        "sort_by": "rating",
        "limit": "3"
    }
    HEADERS = {
        "Authorization": f"Bearer {API_KEY}"
    }

    encoded_params = urllib.parse.urlencode(PARAMS)
    request_url = f"{URL}?{encoded_params}"

    max_retries = 4  # Maximum number of retries
    retry_delay = 0.2  # Time delay (in seconds) between retries

    for attempt in range(max_retries):
        try:
            request = urllib.request.Request(request_url, headers=HEADERS)
            response = urllib.request.urlopen(request)
            response_data = response.read().decode("utf-8")
            data = json.loads(response_data)

            hotels = data["businesses"]
            #coffee_shop = data["businesses"][0]
            suggested_hotels = []
            for hotel in hotels:
                parsed_hotel_url = urllib.parse.urlparse(hotel['url'])
                clean_hotel_url = parsed_hotel_url.scheme + "://" + parsed_hotel_url.netloc + parsed_hotel_url.path
                suggested_hotels.append(f"{hotel['name']} located at {hotel['location']['address1']} "
                                        f"in {hotel['location']['city']}, {hotel['location']['state']}. "
                                        f"Average rating: {hotel['rating']} (based on {hotel['review_count']} reviews) "
                                        f"Yelp URL: {clean_hotel_url}")
            return suggested_hotels
        except urllib.error.HTTPError as e:
            error_description = json.loads(e.read().decode('utf-8'))['error']['description']
            print(f"HTTPError: {e.code} - {e.reason}, Description: {error_description}")
        except urllib.error.URLError as e:
            print(f"URLError: {e.reason}")
        except Exception as e:
            print(f"Error: {str(e)}")

        # If an exception was caught and the loop hasn't ended yet, wait before retrying
        if attempt < max_retries - 1:
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print(f"Failed after {max_retries} attempts. Exiting.")

    return None


In [68]:
from tecton import RequestSource, on_demand_feature_view
from tecton.types import String, Timestamp, Int64, Field, Array

yelp_api_key = os.getenv("YELP_API_KEY") 

request_schema = [Field('recommendation_destination', String)]

transaction_request = RequestSource(schema=request_schema)

@on_demand_feature_view(
    sources=[transaction_request],
    mode='python',
    description="Top rated hotels at the recommended city to check out",
    schema = [Field('hotel_recommendations', Array(String))]
)
def get_best_hotels(transaction_request):
    city_lookup = transaction_request["recommendation_destination"]
    best_hotels = yelp_get_hotels(city_lookup, yelp_api_key)
    return {'hotel_recommendations': best_hotels}

get_best_hotels.validate()

OnDemandFeatureView 'get_best_hotels': Validating 1 dependency.
    Transformation 'get_best_hotels': Successfully validated.
OnDemandFeatureView 'get_best_hotels': Successfully validated.


In [69]:
from tecton import FeatureService

get_best_hotels_fs = FeatureService(
    name="get_best_hotels:v1",
    features=[get_best_hotels]
)
get_best_hotels_fs.validate()

FeatureService 'get_best_hotels:v1': Successfully validated.


In [70]:
event = {"user_id": "demouser123",
         "recommendation_destination": "San Francisco, California"}
training_event = pd.DataFrame([event])

get_best_hotels_df = get_best_hotels_fs.get_historical_features(training_event).to_pandas()


---- Executing stage: Evaluating feature view pipelines                  00:00
---- Executing stage: Computing aggregated features & joining results    00:00
---- Executing stage: Computing ODFV features                            00:00


In [71]:
get_best_hotels_df

,user_id,recommendation_destination,get_best_hotels__hotel_recommendations
0,demouser123,"San Francisco, California",[The Inn San Francisco located at 943 S Van Ne...


In [72]:
from tecton import RequestSource, on_demand_feature_view
from tecton.types import String, Timestamp, Int64, Field

request_schema = [Field('destination_airport', String)]

transaction_request = RequestSource(schema=request_schema)

@on_demand_feature_view(
    sources=[user_home_airport_fv, user_flight_search_fv, transaction_request],
    mode='python',
    description="Generates a flight booking link",
    schema = [Field('booking_url', String)]
)
def get_booking_url(user_home_airport_fv, user_flight_search_fv, transaction_request):
    home_airport = user_home_airport_fv["home_airport_code"]
    destination_airport = transaction_request['destination_airport']
    departure_date = str(user_flight_search_fv['departure_date'])
    return_date = str(user_flight_search_fv['return_date'])

    return {
        'booking_url': (f"https://www.kayak.com/flights/{home_airport}-{destination_airport}/{departure_date}/{return_date}?sort=bestflight_a")
    }
get_booking_url.validate()

OnDemandFeatureView 'get_booking_url': Validating 1 of 3 dependencies. (2 already validated)
    Transformation 'get_booking_url': Successfully validated.
OnDemandFeatureView 'get_booking_url': Successfully validated.


In [73]:
from tecton import FeatureService

hotel_and_flight_service = FeatureService(
    name="hotel_and_flight_service:v1",
    features=[get_best_hotels,
              get_booking_url]
)
hotel_and_flight_service.validate()

FeatureService 'hotel_and_flight_service:v1': Successfully validated.


In [74]:
event = {"user_id": "demouser123",
         "timestamp": datetime.now(),
         "destination_airport": "SIN",
         "recommendation_destination": "Singapore"}
training_event = pd.DataFrame([event])

booking_url_df = hotel_and_flight_service.get_historical_features(training_event, from_source=True).to_pandas()
booking_url_df.style.format({'get_booking_url__booking_url': lambda x: f'<a href="{x}" target="_blank">{x}</a>'})

---- Executing stage: Reading data sources                               00:02
---- Executing stage: Evaluating feature view pipelines                  00:00
---- Executing stage: Computing aggregated features & joining results    00:00
---- Executing stage: Computing ODFV features                            00:00


,user_id,timestamp,destination_airport,recommendation_destination,get_best_hotels__hotel_recommendations,get_booking_url__booking_url
0,demouser123,2023-11-16 09:33:04.325333,SIN,Singapore,"['Hotel Indigo Singapore Katong located at 86 East Coast Rd in Singapore, SG. Average rating: 5.0 (based on 7 reviews) Yelp URL: https://www.yelp.com/biz/hotel-indigo-singapore-katong-singapore', 'AMOY by Far East Hospitality located at 76 Telok Ayer Street in Singapore, SG. Average rating: 5.0 (based on 7 reviews) Yelp URL: https://www.yelp.com/biz/amoy-by-far-east-hospitality-singapore', 'Oasia Hotel Downtown, Singapore located at 100 Peck Seah St in Singapore, SG. Average rating: 5.0 (based on 6 reviews) Yelp URL: https://www.yelp.com/biz/oasia-hotel-downtown-singapore-singapore-2']",https://www.kayak.com/flights/SFO-SIN/2023-12-13/2023-12-27?sort=bestflight_a
